In [14]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\Working_attributes")
from Imports import * 

In [95]:
print_to_excel = True
file_name = 'invest_source_31.10_to_27.11'

Импорт

In [16]:
data_manual = export_from_RISKCUSTOM('''select *
                                     from "RISKACCESS"."sapBwInvestmentReport"
                                     where "reportDate" = TO_DATE('31/10/24', 'DD/MM/YY')
                                     ''')
data_manual.shape

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:92: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


(102413, 38)

In [17]:
data_auto = export_from_RISKCUSTOM('''select *
                                     from "RISKACCESS"."sapBwInvestmentReport"
                                     where "reportDate" = TO_DATE('27/11/24', 'DD/MM/YY')
                                     ''')
data_auto.shape

(495794, 38)

Task

In [18]:
            # Аня, надо сравнить данные в "RISKACCESS"."sapBwInvestmentReport" за 
            # 31/10 - хорошая, сделана вручную, доверяем
            # и 21/11 - автоматическая. За счет чего больше
            # по максимальному количеству полей.

            # # data = export_from_RISKCUSTOM('select * from "RISKACCESS"."sapBwInvestmentReport" fetch first 1 rows only')
            # data.loc[:,'projectName':]
            # data
            # # companyName - есть ли различие между датами по спискам компаний
            # # actStatus	forecastType	generalStatus

In [19]:
data_manual.companyCode.nunique(), data_auto.companyCode.nunique()

(94, 109)

In [48]:
cols_list = ['companyCode','companyName','plantId','plantName','sector','sectorName','wbsElement','wbsElementName','vendorId','vendorName','projectTypeCode','projectProfile','projectId','costElement','costElementName','versionCode','versionName','calendarYear','calendarMonth','currency','amountIndicators']

data_manual['amountIndicators'] = data_manual['amountIndicators'].astype(float)
data_manual['index_copy'] = data_manual.index
data_manual[cols_list] = data_manual[cols_list].astype(str)
data_manual[cols_list] = data_manual[cols_list].apply(lambda x: x.str.strip())

data_auto['amountIndicators'] = data_auto['amountIndicators'].astype(float)
data_auto['index_copy'] = data_auto.index
data_auto[cols_list] = data_auto[cols_list].astype(str)
data_auto[cols_list] = data_auto[cols_list].apply(lambda x: x.str.strip())

In [21]:
merge_data = data_manual.merge(data_auto, how='outer', on=cols_list, suffixes=['_31/10','_27/11'])

In [22]:
merge_data.shape[0]

518656

Компании которых нет в 31

In [23]:
print(merge_data[merge_data['##origin_name_31/10'].isna()].shape[0])

416243


In [24]:
companyCode_list = [x for x in data_auto.companyCode.unique() if x not in data_manual.companyCode.unique()]

In [25]:
data_companies_not_in_manual = merge_data[(merge_data['##origin_name_31/10'].isna())&(merge_data.companyCode.isin(companyCode_list))]
print(data_companies_not_in_manual.shape[0])

618


неравный объем

In [49]:
cols_list.remove('amountIndicators')
data_manual['amountIndicators'] = data_manual['amountIndicators'].astype(float)
data_auto['amountIndicators'] = data_auto['amountIndicators'].astype(float)

count= 0 
for data in [data_auto, data_manual]:
    count += 1
    data = data.groupby(as_index=False, dropna=False, by=cols_list).agg({'amountIndicators':sum, 'index_copy':list})
    if count==1:
        data_auto_group = data
    if count==2:
        data_manual_group = data

merge_values = data_manual_group.merge(data_auto_group, how='left', on=cols_list, suffixes=['_31/10','_27/11'])
merge_values.shape[0]

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_16920\1977855083.py:8: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  data = data.groupby(as_index=False, dropna=False, by=cols_list).agg({'amountIndicators':sum, 'index_copy':list})
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_16920\1977855083.py:8: FutureWarning: The provided callable <built-in function sum> is currently using SeriesGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  data = data.groupby(as_index=False, dropna=False, by=cols_list).agg({'amountIndicators':sum, 'index_copy':list})


102413

In [51]:
merge_values = is_approximately_equal_for_cols(merge_values, 'amountIndicators_31/10', 'amountIndicators_27/11')

In [68]:
not_equal_values = merge_values[merge_values['Diff__more_0.4']==True].reset_index(drop=True)
not_equal_values.head(4)

,companyCode,companyName,plantId,plantName,sector,sectorName,wbsElement,wbsElementName,vendorId,vendorName,...,versionCode,versionName,calendarYear,calendarMonth,currency,amountIndicators_31/10,index_copy_31/10,amountIndicators_27/11,index_copy_27/11,Diff__more_0.4
0,1100,"АО ""СУЭК""",1101,СУЭК - Москва,0A06,"головной офис, СУЭК",C.11010002-A.01.075.01,А/С БЕЛАЗ 7530 ЛУР,1000082614,"ООО ""Промтехснаб""",...,045,Платежи. Прогноз,2024,11,RUU,258683.98,[55477],7957.04,[82613],True
1,1100,"АО ""СУЭК""",1101,СУЭК - Москва,0A06,"головной офис, СУЭК",C.11010002-A.01.075.01,А/С БЕЛАЗ 7530 ЛУР,1000082614,"ООО ""Промтехснаб""",...,075,Платежи бНДС,2024,11,RUU,215570.02,[55478],6630.90,[82614],True
2,1100,"АО ""СУЭК""",1101,СУЭК - Москва,0A06,"головной офис, СУЭК",C.11010002-A.01.076.01,А/С БЕЛАЗ 7530 ЛУР,1000082614,"ООО ""Промтехснаб""",...,045,Платежи. Прогноз,2024,11,RUU,258683.98,[55479],7957.04,[82617],True
3,1100,"АО ""СУЭК""",1101,СУЭК - Москва,0A06,"головной офис, СУЭК",C.11010002-A.01.076.01,А/С БЕЛАЗ 7530 ЛУР,1000082614,"ООО ""Промтехснаб""",...,075,Платежи бНДС,2024,11,RUU,215570.02,[55480],6630.90,[82618],True


In [119]:
merge_values.loc[merge_values['Diff__more_0.4']==False,'currency'].unique()

array(['RUB', 'RUE', 'RUU', 'JPY', 'RUY', 'CNY', 'USD', 'RUY5', 'RUU3',
       'RUE5', 'EUR', 'RUU5', 'RUG', 'RUC', 'RUY3', 'CAD', 'RUK', 'ZAR',
       'CNH', 'INR', 'RUK5', 'KZT'], dtype=object)

In [69]:
wbs_tocheck = not_equal_values.loc[not_equal_values.index==1,'wbsElement'].item()
wbs_tocheck

'C.11010002-A.01.075.01'

компании есть в обоих, но в 21 больше строк чем в 31 (не нулевых)

In [38]:
data_companies_in_manual = merge_data[(merge_data['##origin_name_31/10'].isna())
                                      &(~merge_data.companyCode.isin(companyCode_list))
                                      &(merge_data['amountIndicators']>'1')].sort_values('amountIndicators', ascending=False)
print(data_companies_in_manual.shape[0])

29362


In [108]:
wbsElement_list = [wbs_tocheck]

not_equal_example = data_companies_in_manual.loc[data_companies_in_manual.wbsElement.isin(wbsElement_list), :]
not_equal_example.loc[:,'currency':]

,currency,reportDate_31/10,effectiveFrom_31/10,xlsDate_31/10,loadEventId_31/10,amountIndicators,period_31/10,actStatus_31/10,forecastType_31/10,generalStatus_31/10,...,effectiveFrom_27/11,xlsDate_27/11,loadEventId_27/11,period_27/11,actStatus_27/11,forecastType_27/11,generalStatus_27/11,systemMandant_27/11,periodDate_27/11,index_copy_27/11
3416,RUU,NaT,NaT,NaT,NaN,7957.04,NaN,NaN,NaN,NaN,...,None,None,None,11.2024,E0007,Заказ на поставку,Законтрактовано,BWPCLNT100,2024-11-01,82613.0
3418,RUU,NaT,NaT,NaT,NaN,6630.9,NaN,NaN,NaN,NaN,...,None,None,None,11.2024,E0007,Заказ на поставку,Законтрактовано,BWPCLNT100,2024-11-01,82614.0


In [109]:
data_manual_example = data_manual.loc[data_manual.wbsElement.isin(wbsElement_list),:]
data_manual_example.loc[:,'currency':]

,currency,reportDate,effectiveFrom,xlsDate,loadEventId,amountIndicators,period,actStatus,forecastType,generalStatus,systemMandant,periodDate,index_copy
55477,RUU,2024-10-31,2024-11-07,2024-10-31,298607,258683.98,None,None,None,None,None,2024-11-01,55477
55478,RUU,2024-10-31,2024-11-07,2024-10-31,298607,215570.02,None,None,None,None,None,2024-11-01,55478


In [110]:
data_auto_example = data_auto.loc[data_auto.wbsElement.isin(wbsElement_list),:]
data_auto_example.loc[:,'currency':]

,currency,reportDate,effectiveFrom,xlsDate,loadEventId,amountIndicators,period,actStatus,forecastType,generalStatus,systemMandant,periodDate,index_copy
81800,RUU,2024-11-27,None,None,None,0.00,11.2024,E0007,Заказ на поставку,Законтрактовано,BWPCLNT100,2024-11-01,81800
82613,RUU,2024-11-27,None,None,None,7957.04,11.2024,E0007,Заказ на поставку,Законтрактовано,BWPCLNT100,2024-11-01,82613
82614,RUU,2024-11-27,None,None,None,6630.90,11.2024,E0007,Заказ на поставку,Законтрактовано,BWPCLNT100,2024-11-01,82614
82615,RUU,2024-11-27,None,None,None,0.00,11.2024,E0007,Заказ на поставку,Законтрактовано,BWPCLNT100,2024-11-01,82615


отчего зависят расхождения

In [92]:
cols_isna = (~data_companies_in_manual.isna()).any(axis=0)
cols_isntna = cols_isna[cols_isna==True].index.tolist()

describe_cols = pd.DataFrame({'col_name':cols_isntna, 'n_unique':[data_companies_in_manual[x].nunique() for x in cols_isntna ],'unique':[str(data_companies_in_manual[x].unique().tolist()) for x in cols_isntna ]},)
describe_cols.head(4)

,col_name,n_unique,unique
0,companyCode,77,"['E230', '3D00', 'E110', 'U600', 'E530', '9C00..."
1,companyName,77,"['ООО «ЕХСЗ-2»', 'АО ""Разрез Тугнуйский""', 'ОО..."
2,plantId,190,"['E23A', '3D01', 'E11A', 'U636', 'E53A', '3D02..."
3,plantName,174,"['ООО ЕХСЗ-2', 'р. Тугнуйский', 'Управление', ..."


To excel

In [116]:
# to excel
if print_to_excel == True:
    Output_file = f'{str(date.today())}_{file_name}.xlsx'
    with pd.ExcelWriter(Output_file, engine='openpyxl') as writer:
        # data_companies_not_in_manual.to_excel(writer, index=False, sheet_name='companies_not_in_31.10')
        # data_auto.to_excel(writer, index=True, sheet_name='data_21.11')
        # data_manual.to_excel(writer, index=True, sheet_name='data_31.10')

        not_equal_values.to_excel(writer, index=False, sheet_name='not_equal_amount')

        row = 0
        not_equal_example.to_excel(writer, index=False, sheet_name='example', startrow=row)
        row = row + not_equal_example.shape[0] + 3
        pd.DataFrame({'data 31/10':''},index=[0]).to_excel(writer, index=False, sheet_name='example', startrow=row)
        row +=1
        data_manual_example.to_excel(writer, index=False, sheet_name='example', startrow=row)
        row = row + data_manual_example.shape[0] + 3
        pd.DataFrame({'data 27/11':''},index=[0]).to_excel(writer, index=False, sheet_name='example', startrow=row)
        row +=1
        data_auto_example.to_excel(writer, index=False, sheet_name='example', startrow=row)

        describe_cols.to_excel(writer, index=False, sheet_name='describe_cols')




